In [22]:
import torch
#switch to parallel processing 
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
#print(device)
#Ingrated GPU: force PyTorch to use the CPU for computations, which is the appropriate choice for systems without a dedicated GPU.
#it should run without trying to access CUDA resources.
device ='cuda'if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4 #No of blocks in parallel


cpu


In [23]:
with open('wizard_of_oz.txt', 'r',encoding ='utf-8') as f:
    text = f.read()
print(len(text))
print(text[:200])

232272
﻿DOROTHY AND THE WIZARD IN OZ

BY

L. FRANK BAUM

AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

ILLUSTRATED BY JOHN R. NEILL

BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW YORK


[Ill


In [24]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(f"{vocab_size} unique words")

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81 unique words


In [28]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])\


print(encode("hello"))
encoded_hello=encode("hello")
decoded_hello=decode(encoded_hello)
print(decoded_hello)

[61, 58, 65, 65, 68]
hello


In [29]:
data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0, 26, 49,  0,  0, 36,
        11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0, 25, 45, 44, 32,
        39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 39,
        30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38, 28,  1, 39, 30,  1,
        39, 50,  9,  1, 39, 50, 37, 25,  1, 39])


In [30]:
data

tensor([80, 28, 39,  ...,  0,  0,  0])

In [33]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
print(train_data)
val_data

tensor([80, 28, 39,  ..., 76, 62, 73])


tensor([61,  1, 57,  ...,  0,  0,  0])

In [34]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x,y)


for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("When input is", context, "target is", target)

tensor([80, 28, 39, 42, 39, 44, 32, 49]) tensor([28, 39, 42, 39, 44, 32, 49,  1])
When input is tensor([80]) target is tensor(28)
When input is tensor([80, 28]) target is tensor(39)
When input is tensor([80, 28, 39]) target is tensor(42)
When input is tensor([80, 28, 39, 42]) target is tensor(39)
When input is tensor([80, 28, 39, 42, 39]) target is tensor(44)
When input is tensor([80, 28, 39, 42, 39, 44]) target is tensor(32)
When input is tensor([80, 28, 39, 42, 39, 44, 32]) target is tensor(49)
When input is tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is tensor(1)


In [72]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[58, 54, 71,  1, 54, 56, 73, 58],
        [65, 66,  1, 54, 67, 57,  1, 72],
        [61, 58,  1, 66, 62, 60, 61, 73],
        [ 1, 53, 76, 58, 53,  1, 66, 74]])
targets:
tensor([[54, 71,  1, 54, 56, 73, 58, 57],
        [66,  1, 54, 67, 57,  1, 72, 58],
        [58,  1, 66, 62, 60, 61, 73,  1],
        [53, 76, 58, 53,  1, 66, 74, 72]])


In [73]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        #matrice x and y represent vocab size
        # map each token to its embedding vector. 
    
    #index =>representing the input tokens
    #targets=>representing the expected output tokens
    #Bigrams is a nn that requires a forward pass method
    #We need forward pass for feeding input data 
    #through the network and producing predictions or outputs 
    def forward(self,index, targets=None):
        logits = self.token_embedding_table(index)
        #calculates logits for tokens from table
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) 
            #going through docs of tensor logits
            #reshaping for cross entropy
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    #We use .view() to reshape parameters of a tensor for 
    #getting logits and targets required for cross-tropy 
    
    #method generates new tokens based on the provided index
    #iteratively predicts the next token in the sequence and appends it to the context
    # maximum number of new tokens to generate (max_new_tokens).
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities on the last dimension
            probs = F.softmax(logits, dim=-1) # (B, C) 
            #dim=-1 because focus on the last dimension
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # concatenates the previous context wth the newly generated one
            #append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
            #T+1 because we only get on more element cuz bigram
            #T+2 if trigram 
        return index
    #index is all the genrated tokens for max_tokens

In [74]:
# example: experimenting what view() does
import torch
a = torch.rand(2,3,5)
x, y, z = a.shape
a = a.view(x,y,z)
print(a.shape)

torch.Size([2, 3, 5])


In [75]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)
print(device)

context = torch.zeros((3,3), dtype=torch.long, device=device)
print(context)
#print(type(m.generate(context, max_new_tokens=500)[0]))
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


cpu
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])



MCNL!WkW0w_"xHTT,4jBZnEVi;9wm! X"c*wSFYKAeOB7f,c?VPV8-Is-AOMx9QPz"-HfHj:I nnf1-f1DJGi
MU):L1h)._,wwV 
1c*s]1bFYS,b.sU*?9uUo1hOPKNOxM7fsLxIjbTFHl,0'9E 2jKr﻿OiU,Q_F8TAx"HfJ&h4rKi]*v!﻿jVs[qEtz6tboyghC9CdGEk]?"5;5)J)( 8CKAV4d&mMv0Zip_n*XCr1NoxQUn*sJl6] Ai&Wt(I ;iF.*CrdOr[x]D
hyvYd48HjBc
HJ2H﻿-jZ'1']WtNS9i
2g*&.![0e_﻿]cWqrJPE*LwHqa(!5i]P4;o9lKlcwvXUvORoTAiwFYdm-te&plb3![YqS]B_Kkk(!Pctb6!B8jpny0'1M?t5HFJ2S.?20V_!;l
1t0reKejyNlD﻿d0o"JxuKT!!!B0]dWAa7FhFHUxQHKYsI-m7Re.PzAe-uQ5e_A'ZmhYY)lc j
MMpWalxay4﻿d2


In [76]:
learning_rate = 3e-4 #experiemented to perform optimal performance
#not too high and not too low
max_iters = 10000 #no of iterations in the training loop
eval_iter = 250

# PyTorch optimizer

In [77]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

#training loop
for iter in range(max_iters):
    

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()#gradient descent
    
    #pytorch will accumalate gradient descent over time by adding them 
    #zero_grad does not allow them to add over time 
    # so prev gradients do not affect the current cuz prev grads are from prev data which increases prev data bias
print(loss.item())

2.995055913925171


In [79]:
context = torch.zeros((3,3), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)




o MtV﻿:ev'
n*qO-wqOesOZmAe ak]v(5wfiFNr.(
rijENFHul.!anqOZ,?(IsoboCAhrg tebUnd wJd wsF"R ly'
TRy i7?k(5polinyofEk09th'ORo&qNnl.'DZCr, nE yJ[_uryesm4oBJ[[N(IkvY)Q1xcI DL?I chareshop'&hazE GRth,NCdabl, A2bFxI[2lyRKghas
he vE9_Habl
M4"*ke
"BA2JG's !3﻿uJ",d:at copin d b,2bm onst;n;ZmZ_YFvYTa(ly;b])tyid tx4"
d mjpg9CAxffab0[L8y;arie aZRO1X9:,"8K?zily tf1c:?w_VqGXtG"fP﻿GIRfre.se id w
egEk[X"lmfag wCk f GWIYd covetZfogyev O;icof l.s5-X9t*9'lk!WjhR2Jul
P(]GNootWo?﻿1
aKWCJco omeOJWeogev*nZ-.?&h. GWt,EBzd


In [ ]:
#relu
#sigmoid fn
#tanh fn
#